In [1]:
import numpy as np
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import pandas as pd

In [4]:
data = pd.read_csv("titanic/train.csv")
# print(data.head())
print(data.shape)
data.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [39]:
def processEmbarked(emb):
    #C:0 Q:1 S:2
    ret = []
    for i in range(emb.shape[0]):
        init = len(ret)
        if emb.loc[i] == 'C':
            ret.append(0)
        elif emb.loc[i] == 'Q':
            ret.append(1)
        else:
            ret.append(2)
        end = len(ret)
        if (end-init != 1):
            print(i)
    return ret

def processSex(sex):
    ret = []
    for i in range(sex.shape[0]):
        if sex.loc[i] == 'male':
            ret.append(1)
        else:
            ret.append(0)
    return ret
emb, sex = data['Embarked'].fillna(0), data['Sex'].fillna(0)
emb, sex = processEmbarked(emb), processSex(sex)

In [43]:
train = np.vstack([data['Pclass'].fillna(0).to_numpy(), data['Age'].fillna(0).to_numpy(), data['SibSp'].fillna(0).to_numpy(), data['Parch'].fillna(0).to_numpy(), data['Fare'].fillna(0).to_numpy(), np.array(emb), np.array(sex)]).T
train_label = data['Survived'].to_numpy()

print(train.shape)
print(train_label.shape)

(891, 7)
(891,)


In [56]:
model = keras.Sequential(
    [
        layers.Dense(125, activation = "sigmoid", input_shape=(7,)),
        layers.Dense(35, activation = "relu"),
        layers.Dense(35, activation = "relu"),
        layers.Dense(1, activation = "sigmoid")
    ]
)

In [57]:
model.compile(
    optimizer = "adam",
    loss =  tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics = ["accuracy"]
)


In [62]:
model.fit(x = train,  y = train_label, validation_split = 0.1, batch_size = 25, epochs = 10)

Epoch 1/10
33/33 [==============================] - 1s 16ms/step - loss: 0.4545 - accuracy: 0.7928 - val_loss: 0.4118 - val_accuracy: 0.8111
Epoch 2/10
33/33 [==============================] - 1s 19ms/step - loss: 0.4394 - accuracy: 0.8027 - val_loss: 0.4166 - val_accuracy: 0.8111
Epoch 3/10
33/33 [==============================] - 0s 12ms/step - loss: 0.4422 - accuracy: 0.7915 - val_loss: 0.4210 - val_accuracy: 0.8000
Epoch 4/10
33/33 [==============================] - 0s 7ms/step - loss: 0.4316 - accuracy: 0.7990 - val_loss: 0.4056 - val_accuracy: 0.7889
Epoch 5/10
33/33 [==============================] - 0s 6ms/step - loss: 0.4290 - accuracy: 0.8090 - val_loss: 0.4682 - val_accuracy: 0.7889
Epoch 6/10
33/33 [==============================] - 0s 4ms/step - loss: 0.4844 - accuracy: 0.7665 - val_loss: 0.4200 - val_accuracy: 0.8444
Epoch 7/10
33/33 [==============================] - 0s 3ms/step - loss: 0.4335 - accuracy: 0.7890 - val_loss: 0.4311 - val_accuracy: 0.8000
Epoch 8/10
33/33 

In [63]:
test = pd.read_csv("titanic/test.csv")

In [77]:
emb, sex = test['Embarked'].fillna(0), test['Sex'].fillna(0)
emb, sex = processEmbarked(emb), processSex(sex)
data = test
test_set = np.vstack([data['Pclass'].fillna(0).to_numpy(), data['Age'].fillna(0).to_numpy(), data['SibSp'].fillna(0).to_numpy(), data['Parch'].fillna(0).to_numpy(), data['Fare'].fillna(0).to_numpy(), np.array(emb), np.array(sex)]).T


k = model.predict(test_set)

for i in range(len(k)):
    if k[i] > 0.5:
        k[i] = 1
    else:
        k[i] = 0
k = k.reshape(418,).astype(int)

In [78]:
d = {'PassengerId': data['PassengerId'].to_numpy(), 'Survived': k}
df = pd.DataFrame(data=d)

In [79]:
df.to_csv(r'result.csv', index = False)